## 6.2 Das CG-Verfahren für symmetrisch positiv definite Gleichungssysteme

In [ ]:
import numpy as np
import time

**Implementierung 6.1: CG-Verfahren**

Wir implementieren das CG-Verfahren mit Hilfe der `numpy` Routinen für Matrix-Vektor- und Skalarprodukte. Dabei achten wir darauf, dass wir jedes Produkt nur einmal pro Iteration berechnen und wir das Ergebnis wieder verwenden.

In [ ]:
def cg_verfahren(A, b, x, it=100, tol=1e-5):
    x = x.copy()
    r = b - A.dot(x)
    d = r.copy()
    nrm_r2 = r.dot(r)
    tol = tol**2
    
    for i in range(1, it + 1):
        if nrm_r2 < tol:
            break
        Ad = A.dot(d)
        dAd = d.dot(Ad)
        alpha = nrm_r2 / dAd
        x[:] += alpha * d
        r[:] -= alpha * Ad
        nrm_r2_neu = r.dot(r)
        
        beta = nrm_r2_neu / nrm_r2
        d[:] = r + beta * d
        nrm_r2 = nrm_r2_neu
    else:
        print(f'Das CG-Verfahren ist nach {i} Iterationen nicht konvergiert.')
    return x, i

Um das Verfahren mit den bisherigen Iterationsverfahren zum Lösen linearer Gleichungssysteme zu vergleichen, betrachten wir zunächst wieder die Modellmatrix aus Beispiel 3.50

In [ ]:
for i in range(1, 11):
    m = i * 10
    n = m**2
    N = np.diag(np.ones(m - 1), 1) + np.diag(np.ones(m - 1), -1)
    B = 4 * np.eye(m) - N
    A = np.kron(np.eye(m), B) - np.kron(N, np.eye(m))
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = cg_verfahren(A, b, x0, it=int(1e6), tol=1e-6)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'CG-Verfahren: n = {n:05d}, Schritte = {m:05d} Zeit = {t:07.4f}sec, res = {res:4.2e}')

Da die Implementierung des CG-Verfahrens keine Zugriffe auf Teil-Matrizen oder Vektoren benötigt, ist unsere Implementierung sogar deutlich effizienter als das SOR-Verfahren, selbst bei optimaler Wahl des Relaxationsparameter.

## 6.3 Vorkonditioniertes CG-Verfahren

**CG-Verfahren mit Jacobi Vorkonditionierung**

Um das vorkonditionierte CG-Verfahren zu implementieren, müssen wir eine Wahl für die Vorkonditionierung $P\approx A^{-1}$ treffen. Die einfachste Wahl ist hier die Jacobi-Vorkonditionierung mit $P=D^{-1}$, wobei $D$ der Diagonalteil von A ist.

In [ ]:
def cg_verfahren_jacobi_vorkon(A, b, x, it=100, tol=1e-5):
    x = x.copy()
    r = b.copy() - A.dot(x)
    P_inv = 1 / np.diag(A)
    p = P_inv * r.copy()
    d = p.copy()
    rp = r.dot(p)
    tol = tol**2
    
    for i in range(1, it + 1):
        if abs(rp) < tol:
            break
        Ad = A.dot(d)
        alpha = rp / d.dot(Ad)
        x[:] += alpha * d
        r[:] -= alpha * Ad
        p[:] = P_inv * r
        rp_neu = r.dot(p)
        beta = rp_neu / rp
        d[:] = p + beta * d
        rp = rp_neu
    else:
        print(f'Das Jacobi-vorkonditionierte CG-Verfahren ist nach {i} Iterationen nicht konvergiert.')
    return x, i

#### Beispiel 6.10

Angewandt auf die Modellmatrix aus Beispiel 3.50 ergibt das Jacobi-vorkonditionierte CG-Verfahren folgende Ergebnisse.

In [ ]:
for i in range(1, 11):
    m = i * 10
    n = m**2
    N = np.diag(np.ones(m - 1), 1) + np.diag(np.ones(m - 1), -1)
    B = 4 * np.eye(m) - N
    A = np.kron(np.eye(m), B) - np.kron(N, np.eye(m))
    b = np.ones(n)
    x0 = np.zeros(n)
    
    t = time.perf_counter()
    x, m = cg_verfahren_jacobi_vorkon(A, b, x0, it=int(1e6), tol=1e-6)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Jacobi-vorkonditionierte CG-Verfahren: n = {n:05d}, Schritte = {m:05d} Zeit = {t:07.4f}sec, res = {res:4.2e}')    

Die Jacobi-Vorkonditionierung hat uns in diesem Fall also nicht wesentlich geholfen. Um die SSOR-Vorkonditionierung zu implementieren, widmen wir uns zunächst dem vorkonditionierten CG-Verfahren mit einem allgemeinen Vorkonditionerer $P$. Dabei verstecken wir das Anwenden von $P^{-1}$ in `np.linalg.solve(P, r)`.

In [ ]:
def cg_verfahren_vorkon(A, P, b, x, it=100, tol=1e-5):
    x = x.copy()
    r = b.copy() - A.dot(x)
    p = np.linalg.solve(P, r)
    d = p.copy()
    rp = r.dot(p)
    tol = tol**2
    
    for i in range(1, it + 1):
        if abs(rp) < tol:
            break
        Ad = A.dot(d)
        alpha = rp / d.dot(Ad)
        x[:] += alpha * d
        r[:] -= alpha * Ad 
        p[:] = np.linalg.solve(P, r)
        rp_neu = r.dot(p)

        beta = rp_neu / rp
        d[:] = p + beta * d
        rp = rp_neu
    else:
        print(f'Das vorkonditionierte CG-Verfahren ist nach {i} Iterationen nicht konvergiert.')
    return x, i

Angewandt auf die Modellmatrix ergibt sich damit mit der optimalen Wahl von $\omega=2 - \frac{2\pi}{\sqrt{n}}$ die Ergebnisse.

In [ ]:
for i in range(1, 9):
    m = i * 10
    n = m**2
    N = np.diag(np.ones(m - 1), 1) + np.diag(np.ones(m - 1), -1)
    B = 4 * np.eye(m) - N
    A = np.kron(np.eye(m), B) - np.kron(N, np.eye(m))
    b = np.ones(n)
    x0 = np.zeros(n)
    b = np.ones(n)
    x0 = np.zeros(n)
    
    omega = 2 - 2 * np.pi / np.sqrt(n)
    D = np.diag(np.diag(A))
    D1 = np.diag(1 / np.diag(A))
    L = np.tril(A, -1)
    R = np.triu(A, 1)
    P = (D + omega * L) @ D1 @ (D + omega * R)
    
    t = time.perf_counter()
    x, m = cg_verfahren_vorkon(A, P, b, x0, it=int(1e6), tol=1e-6)
    t = time.perf_counter() - t
    
    res = np.linalg.norm(b - np.dot(A, x))
    print(f'Das SSOR-vorkonditionierte CG-Verfahren: n = {n:04d}, Schritte = {m:05d} Zeit = {t:07.4f}sec, res = {res:4.2e}')

Die Konvergenz hat sich also wesentlich verbessert! Die Anzahl der notwendigen Schritte wächst auch nur sehr langsam. Allerdings ist das Lösen des Systems `pneu = np.linalg.solve(P, rneu)` hier wesentlich teurer, sodass die Rechenzeit jedes Schrittes aufwändiger ist. Es zeigt sich also, dass die Kosten des Anwenden des Vorkonditionierers eine wesentliche Rolle bei der Wahl spielten.